In [2]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# Read the CSV file
data_path = 'cleaned_tele_data.csv'
df = pd.read_csv(data_path)

In [15]:
# Ensure the 'ID' column is numeric and drop invalid rows
df['ID'] = pd.to_numeric(df['ID'], errors='coerce')  # Convert invalid IDs to NaN
df = df.dropna(subset=['ID'])  # Drop rows where 'ID' is NaN
df['ID'] = df['ID'].astype(int)  # Ensure IDs are integers

/tmp/ipykernel_163099/3445115727.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID'] = df['ID'].astype(int)  # Ensure IDs are integers


In [16]:
# PostgreSQL connection details
db_params = {
    'dbname': 'tele_data',
    'user': 'kidsu',
    'password': 'admin@123',
    'host': 'localhost',
    'port': '5432'
}

# Establish the connection
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [ ]:
# Create a table for the data 
create_table_query = '''
CREATE TABLE IF NOT EXISTS tele_data (
    channel_title TEXT,
    channel_username TEXT,
    id BIGINT PRIMARY KEY,
    message TEXT,
    date TIMESTAMP,
    media_path TEXT
);
'''
cur.execute(create_table_query)

try:
    # Insert data into the PostgreSQL table
    for index, row in df.iterrows():
        try:
            insert_query = sql.SQL('''
            INSERT INTO tele_data (channel_title, channel_username, id, message, date, media_path)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING;
            ''')
            cur.execute(insert_query, (row['Channel Title'], row['Channel Username'], row['ID'],
                                       row['Message'], row['Date'], row['Media Path']))
        except Exception as e:
            # If an error occurs on an individual insert, log it and continue
            print(f"Error inserting row {index}: {e}")
    
    # Commit the transaction after all rows are inserted
    conn.commit()

except Exception as e:
    # If any error occurs, rollback the transaction
    print(f"Transaction failed, rolling back: {e}")
    conn.rollback()

finally:
    # Close the cursor and connection
    cur.close()
    conn.close()

print("Data storage process completed!")